In [1]:
'''
Date: 15/Aug/2022
Auther: Yike Xie
Content: Prepare files to Bursa
'''

'\nDate: 15/Aug/2022\nAuther: Yike Xie\nContent: Prepare files to Bursa\n'

# DEGs of megakaryocytes

In [2]:
import os

import numpy as np
import pandas as pd

import anndata
import scanpy as sc

from collections import defaultdict
import anndataks

In [3]:
save_tables = '/home/yike/phd/dengue/docs/paper_draft/tabels/'

### SD vs D

In [4]:
ct_pair = pd.read_csv(save_tables + 'ct_pair.tsv', sep='\t', index_col=['cell_subtype', 'gene'])
# ct_pair.loc['megakaryocytes'].to_csv(save_tables + 'mega_pairwise_SD_vs_D.tsv', sep='\t')

### SD vs non_SD (D & DWS)

In [3]:
import sys
sys.path.append('/home/yike/phd/dengue/dengue_children/paper_figures/final/modules') 
import YK_util, optimized_pair_comparison, functions_CCC_figure_fra_pair
from YK_util import *
from optimized_pair_comparison import *
from functions_CCC_figure_fra_pair import *

In [4]:
path = '/home/yike/phd/dengue/data/mergedata_luca_NKT_202206.h5ad'
adata = sc.read_h5ad(path)
adata_kid = subsetdata(adata)
adata_kid = normalizedata(adata_kid, log1p=True) # log1p = 2
adata_kid = removegenes(adata_kid)
adata_kid = adata_kid[(adata_kid.obs.cell_subtype_new != 'doublets') & 
                        (adata_kid.obs.cell_subtype_new != 'unknown')].copy()

sc.pp.filter_genes(adata_kid, min_cells=3)

/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)


In [5]:
#rename and reorder some celltypes 
adata_kid = adata_kid.copy()
reorg_data(adata_kid) # 'cell_type_new', 'cell_subtype_new'

/home/yike/phd/dengue/dengue_children/paper_figures/final/modules/YK_util.py:172: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  adata.obs['cell_type_new'].cat.reorder_categories(group_order_primary, inplace=True)
/home/yike/phd/dengue/dengue_children/paper_figures/final/modules/YK_util.py:204: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  adata.obs['cell_subtype_new'].cat.reorder_categories(group_order_secondary, inplace=True)


In [8]:
ct_obs='cell_type_new'
cd_SD=['S_dengue']
cd_D=['Dengue', 'DWS']
cell_types=[['megakaryocytes']]

pair_res = pair_comparison(adata_kid, ct_obs, 
                           cd_SD, cd_D, 
                           cell_types, gene_cut_off=False, log1p=2)

pair_res['pair_res'].to_csv(save_tables + 'SD_vs_non/' + 'mega_pair.tsv')
pair_res['log_FCs'].to_csv(save_tables + 'SD_vs_non/' + 'mega_log2FC.tsv')

In [12]:
# combine pairwise comparison with fra and avg
ct_obs = 'cell_type_new'
cts = ['megakaryocytes']
cds = {'SD': ['S_dengue'], 'N': ['dengue', 'DWS']}

fra_avg = fra_avg(adata_kid, ct_obs, cts, cds, log1p=2)

pair = combination(pair_res['pair_res'], fra_avg['fra'], fra_avg['avg'])

/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)


In [33]:
# ks test to get log2 fold change

ct_obs = 'cell_type_new'
cts = ['megakaryocytes']
cd_SD=['S_dengue']
cd_D=['dengue', 'DWS']
ks_res = ks(adata_kid, cts, ct_obs, cd_SD, cd_D, log1p=2)
ks_res = ks_res.reset_index().set_index(['cell_type', 'gene'])

pair = pd.concat([pair, ks_res.loc[pair.index][['log2_fold_change', 'statistic', 'pvalue']]], axis=1)

/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)
/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)


In [51]:
# adjust pvalue

from statsmodels.stats.multitest import multipletests

cts = ['megakaryocytes']

ct_data = pd.DataFrame([])
for cell_type in cts:
    data_ct = pair.loc[cell_type]
    adjust_pvalue = multipletests(data_ct['pvalue'], alpha=0.05, method='fdr_bh')[1] # pvals_corrected
    data_ct['pvalue_corrected'] = adjust_pvalue
    data_ct['cell_subtype'] = cell_type
    data_ct.reset_index(inplace=True)
    ct_data = pd.concat([ct_data, data_ct])
    
    
ct_data.to_csv(save_tables + 'SD_vs_non/' + 'mega_pairwise_severe_vs_non.tsv', sep='\t', index=False)

<ipython-input-51-fbc3b9778281>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ct['pvalue_corrected'] = adjust_pvalue
<ipython-input-51-fbc3b9778281>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ct['cell_subtype'] = cell_type


# cytokine list & cytokine interactions

In [24]:
cyto_df = pd.read_csv('/home/yike/phd/dengue/data/interaction_source_file/cytokine_list.csv', sep=',')
cytos = cyto_df.astype('str').values

cyto_ls = np.unique(cytos).tolist()
cyto_ls.remove('nan')

In [91]:
ls = []
for cyto in cyto_ls:
    if '/' in cyto:
        ls += cyto.split('/')
        
    else:
        ls.append(cyto)

In [84]:
fn_int='/home/yike/phd/dengue/data/interaction_source_file/inters_YK_20220324.tsv'
interactions = pd.read_csv(fn_int, sep='\t', dtype={'is_stimulation': 'object',
                                             'is_inhibition': 'object',
                                             'consensus_direction': 'object',
                                             'consensus_stimulation': 'object',
                                             'consensus_inhibition': 'object',
                                             'database_intercell_source': 'object',
                                             'transmitter_intercell_source': 'object',
                                             'receiver_intercell_source': 'object',
                                             'secreted_intercell_source': 'object',
                                             'plasma_membrane_transmembrane_intercell_source': 'object',
                                             'plasma_membrane_peripheral_intercell_source': 'object',
                                             'database_intercell_target': 'object',
                                             'transmitter_intercell_target': 'object',
                                             'receiver_intercell_target': 'object',
                                             'secreted_intercell_target': 'object',
                                             'plasma_membrane_transmembrane_intercell_target': 'object',
                                             'plasma_membrane_peripheral_intercell_target': 'object'})

In [93]:
cyto_interactions = interactions[interactions['genesymbol_intercell_target'].isin(ls) | 
                                 interactions['genesymbol_intercell_source'].isin(ls)]

In [94]:
cyto_interactions.to_csv(save_tables + 'SD_vs_non/' + 'cyto_interactions.tsv', sep='\t', index=False)

In [95]:
cyto_interactions

,source,target,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,dip_url,curation_effort,references,...,category_source_intercell_target,uniprot_intercell_target,genesymbol_intercell_target,entity_type_intercell_target,consensus_score_intercell_target,transmitter_intercell_target,receiver_intercell_target,secreted_intercell_target,plasma_membrane_transmembrane_intercell_target,plasma_membrane_peripheral_intercell_target
17,P48061,P04899,True,False,True,True,False,NaN,0.0,NaN,...,resource_specific,P04899,GNAI2,protein,1.0,False,True,False,False,False
97,P22301,P01588,False,True,True,False,True,NaN,0.0,NaN,...,resource_specific,P01588,EPO,protein,1.0,False,True,True,False,False
98,P01579,P01588,False,True,True,False,True,NaN,0.0,NaN,...,resource_specific,P01588,EPO,protein,1.0,False,True,True,False,False
103,P01584,P01588,False,True,True,False,True,NaN,0.0,NaN,...,resource_specific,P01588,EPO,protein,1.0,False,True,True,False,False
105,P08887,P01588,True,False,True,True,False,NaN,0.0,NaN,...,resource_specific,P01588,EPO,protein,1.0,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37403,P55774,Q9BZ71,False,False,False,False,False,NaN,2.0,InnateDB:21481794;connectomeDB2020:24218476,...,resource_specific,Q9BZ71,PITPNM3,protein,2.0,False,True,False,False,False
37469,Q6UXB2,Q9HC97,False,False,False,False,False,NaN,2.0,CellTalkDB:25411203;connectomeDB2020:25411203,...,resource_specific,Q9HC97,GPR35,protein,7.0,False,True,False,True,False
37521,Q8NHW4,Q96P66,False,False,False,False,False,NaN,0.0,NaN,...,resource_specific,Q96P66,GPR101,protein,6.0,False,True,False,True,False
37522,Q8NHW4,O15173,False,False,False,False,False,NaN,0.0,NaN,...,resource_specific,O15173,PGRMC2,protein,2.0,False,True,False,False,False


# mega DEGs

In [5]:
mega_pair = ct_pair.loc['megakaryocytes']

In [12]:
DEGs_df = mega_pair[((mega_pair['log2_fold_change'] <= -1) | (mega_pair['log2_fold_change'] >= 1)) &
                    (mega_pair['pvalue_corrected'] < 0.05)]

In [19]:
up_DEGs_df = mega_pair[(mega_pair['med_pair'] >= 1) & 
                       (mega_pair['S_fra'] >= 0.02) &
                    (mega_pair['fra_pair'] >= 39/56)]

In [25]:
up_cyto = [i for i in cyto_ls if i in up_DEGs_df.index]

up_DEGs_df.loc[up_cyto]

,med_pair,fra_pair,neg_fra_pair,S_fra,NS_fra,S_avg,NS_avg,log2_fold_change,statistic,pvalue,pvalue_corrected
gene,,,,,,,,,,,


In [22]:
down_DEGs_df = mega_pair[(mega_pair['med_pair'] <= -1) & 
                         (mega_pair['NS_fra'] >= 0.02) &
                    (mega_pair['neg_fra_pair'] >= 39/56)]

In [26]:
down_cyto = [i for i in cyto_ls if i in down_DEGs_df.index]

down_DEGs_df.loc[down_cyto]

,med_pair,fra_pair,neg_fra_pair,S_fra,NS_fra,S_avg,NS_avg,log2_fold_change,statistic,pvalue,pvalue_corrected
gene,,,,,,,,,,,
CXCL3,-1.337524,0.083333,0.75,0.036364,0.160714,0.229430,1.809391,-1.192268,0.142532,0.571410,1.0
LTB,-1.476773,0.083333,0.75,0.036364,0.178571,0.255297,2.063767,-1.287277,0.142208,0.574271,1.0
